In [27]:
# import necessary libraries and packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

I need to join some of the respective datasets together. But fitst lets load them and check that they can be matched up.

In [28]:
# First i think join driver name (in drivers.csv) to driver id (on results.csv). This will make it less confusing. I then want to join 
# constructor name (in constructors.csv) to constructor id (in results.csv). Finally I want to join race id (in races.csv)
# to the overall dataset. This will give me all the information I need in one dataframe for now.

# Load results data
results = pd.read_csv('data_raw/results.csv')

# Load driver metadata, selecting and renaming to avoid name collisions
drivers = pd.read_csv('data_raw/drivers.csv')[['driverId', 'forename', 'surname', 'nationality']]
# create new column for full name
drivers['driverName'] = drivers['forename'] + ' ' + drivers['surname']
drivers = drivers[['driverId', 'driverName', 'nationality']]

# Merge driver info into results (avoid duplicate columns by using merge)
overall = results.merge(drivers, on='driverId', how='inner')

# Load constructor and race metadata, selecting and renaming to avoid name collisions (this is all we need from these two csvs)
constructors = pd.read_csv('data_raw/constructors.csv')[['constructorId', 'name']].rename(columns={'name': 'constructorName'})
races = pd.read_csv('data_raw/races.csv')[['raceId', 'year', 'round', 'name']].rename(columns={'name': 'raceName'})

# Merge constructors and races into the overall dataset
overall = overall.merge(constructors, on='constructorId', how='left').merge(races, on='raceId', how='left')

display(overall)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,rank,fastestLapTime,fastestLapSpeed,statusId,driverName,nationality,constructorName,year,round,raceName
0,1,18,1,1,22,1,1,1,1,10.0,...,2,1:27.452,218.300,1,Lewis Hamilton,British,McLaren,2008,1,Australian Grand Prix
1,2,18,2,2,3,5,2,2,2,8.0,...,3,1:27.739,217.586,1,Nick Heidfeld,German,BMW Sauber,2008,1,Australian Grand Prix
2,3,18,3,3,7,7,3,3,3,6.0,...,5,1:28.090,216.719,1,Nico Rosberg,German,Williams,2008,1,Australian Grand Prix
3,4,18,4,4,5,11,4,4,4,5.0,...,7,1:28.603,215.464,1,Fernando Alonso,Spanish,Renault,2008,1,Australian Grand Prix
4,5,18,5,1,23,3,5,5,5,4.0,...,1,1:27.418,218.385,1,Heikki Kovalainen,Finnish,McLaren,2008,1,Australian Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26754,26760,1144,825,210,20,14,16,16,16,0.0,...,1,1:25.637,222.002,11,Kevin Magnussen,Danish,Haas F1 Team,2024,24,Abu Dhabi Grand Prix
26755,26761,1144,859,215,30,12,17,17,17,0.0,...,12,1:28.751,214.212,5,Liam Lawson,New Zealander,RB F1 Team,2024,24,Abu Dhabi Grand Prix
26756,26762,1144,822,15,77,9,\N,R,18,0.0,...,19,1:29.482,212.462,130,Valtteri Bottas,Finnish,Sauber,2024,24,Abu Dhabi Grand Prix
26757,26763,1144,861,3,43,20,\N,R,19,0.0,...,17,1:29.411,212.631,5,Franco Colapinto,Argentinian,Williams,2024,24,Abu Dhabi Grand Prix


This dataframe looks good! much easier to navigate. Not sure why it starts at 2008 so ill order it based on year.

In [29]:
# Sort by year, round and numeric finishing order
overall = overall.sort_values(by=['year', 'round', 'positionOrder']).set_index('raceId')
overall

,resultId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,...,rank,fastestLapTime,fastestLapSpeed,statusId,driverName,nationality,constructorName,year,round,raceName
raceId,,,,,,,,,,,,,,,,,,,,,
833,20025,642,51,2,1,1,1,1,9.0,70,...,\N,\N,\N,1,Nino Farina,Italian,Alfa Romeo,1950,1,British Grand Prix
833,20026,786,51,3,2,2,2,2,6.0,70,...,\N,\N,\N,1,Luigi Fagioli,Italian,Alfa Romeo,1950,1,British Grand Prix
833,20027,686,51,4,4,3,3,3,4.0,70,...,\N,\N,\N,1,Reg Parnell,British,Alfa Romeo,1950,1,British Grand Prix
833,20028,704,154,14,6,4,4,4,3.0,68,...,\N,\N,\N,12,Yves Cabantous,French,Talbot-Lago,1950,1,British Grand Prix
833,20029,627,154,15,9,5,5,5,2.0,68,...,\N,\N,\N,12,Louis Rosier,French,Talbot-Lago,1950,1,British Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,26760,825,210,20,14,16,16,16,0.0,57,...,1,1:25.637,222.002,11,Kevin Magnussen,Danish,Haas F1 Team,2024,24,Abu Dhabi Grand Prix
1144,26761,859,215,30,12,17,17,17,0.0,55,...,12,1:28.751,214.212,5,Liam Lawson,New Zealander,RB F1 Team,2024,24,Abu Dhabi Grand Prix
1144,26762,822,15,77,9,\N,R,18,0.0,30,...,19,1:29.482,212.462,130,Valtteri Bottas,Finnish,Sauber,2024,24,Abu Dhabi Grand Prix


This is now a neat dataframe sorted by year and in order of round and position that the drivers finished in the race

## How consistently do people overperform in qulifying vs race. who are the biggest overperformers?

we need to make a column for positions gained.

In [30]:
# check that the columns are integers
print(overall['positionOrder'].dtype)
print(overall['grid'].dtype)

int64
int64


In [31]:
# positions gained = grid - finishing order; handle unknown grid (0) as NaN
# use np.where() to be the value arg in .insert(). we want to insert after 'positionOrder' column.
overall.insert(6, 'positions_gained', np.where(overall['grid'] > 0, overall['grid'] - overall['positionOrder'], np.nan))
overall

,resultId,driverId,constructorId,number,grid,position,positions_gained,positionText,positionOrder,points,...,rank,fastestLapTime,fastestLapSpeed,statusId,driverName,nationality,constructorName,year,round,raceName
raceId,,,,,,,,,,,,,,,,,,,,,
833,20025,642,51,2,1,1,0.0,1,1,9.0,...,\N,\N,\N,1,Nino Farina,Italian,Alfa Romeo,1950,1,British Grand Prix
833,20026,786,51,3,2,2,0.0,2,2,6.0,...,\N,\N,\N,1,Luigi Fagioli,Italian,Alfa Romeo,1950,1,British Grand Prix
833,20027,686,51,4,4,3,1.0,3,3,4.0,...,\N,\N,\N,1,Reg Parnell,British,Alfa Romeo,1950,1,British Grand Prix
833,20028,704,154,14,6,4,2.0,4,4,3.0,...,\N,\N,\N,12,Yves Cabantous,French,Talbot-Lago,1950,1,British Grand Prix
833,20029,627,154,15,9,5,4.0,5,5,2.0,...,\N,\N,\N,12,Louis Rosier,French,Talbot-Lago,1950,1,British Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,26760,825,210,20,14,16,-2.0,16,16,0.0,...,1,1:25.637,222.002,11,Kevin Magnussen,Danish,Haas F1 Team,2024,24,Abu Dhabi Grand Prix
1144,26761,859,215,30,12,17,-5.0,17,17,0.0,...,12,1:28.751,214.212,5,Liam Lawson,New Zealander,RB F1 Team,2024,24,Abu Dhabi Grand Prix
1144,26762,822,15,77,9,\N,-9.0,R,18,0.0,...,19,1:29.482,212.462,130,Valtteri Bottas,Finnish,Sauber,2024,24,Abu Dhabi Grand Prix
